In [ ]:
# Some utils
from datetime import datetime
import re
import hashlib

def hash_for_obj(obj):
    return hashlib.md5(str(obj).encode('utf-8')).hexdigest()


def now_in_string():
    return datetime.now().strftime("%H:%M:%S")


def get_type_and_tribunal(long_name):
    """
    Parses the name of each tribunal, and returns the name and the speciality
    """
    result = re.search('([\s\w.]*) - ([\w]*)', long_name)
    return result.groups()

In [ ]:
import urllib.parse
import json

import requests
from bs4 import BeautifulSoup


def load_json_file(path):
    try:
        with open(path) as json_file:
            return json.load(json_file)
    except FileNotFoundError:
        return None
    
def save_to_json_file(path, dictionary):
    with open(path, 'w') as outfile:
        json.dump(dictionary, outfile)


def get_all_specialties():
    CACHE_FILE = 'cache/specialities.json'
    cache = load_json_file(CACHE_FILE)
    if cache:
        return cache
    
    DOCENTES_URL = "https://ceice.gva.es/auto/Actas/"
    r = requests.get(DOCENTES_URL)
    bs_content = BeautifulSoup(r.text, 'lxml')
    table = bs_content.find('table', attrs={'id': 'indexlist'})
    rows = table.findChildren('tr')
    
    structured_rows = []
    for row in rows:
        if not row.get('class')[0] in ['even', 'odd']:
            continue
        cols = row.findChildren('td')
        name = cols[1].a.get_text().strip().replace('.', '').replace('/', '')
        structured = {
            'name': name,
            'link': urllib.parse.urljoin(DOCENTES_URL, cols[1].a.get('href')).strip(),
            'modified': cols[2].get_text().strip(),
        }
        structured_rows.append(structured)

    save_to_json_file(CACHE_FILE, structured_rows)
    return structured_rows

specialties = get_all_specialties()

In [ ]:
import os
from pathlib import Path
from slugify import slugify


def get_all_tribunals(speciality_row):        
    slug = slugify(speciality_row['name'])
    CACHE_FILE = f'cache/tribunals{slug}.json'
    
    cache = load_json_file(CACHE_FILE)
    if cache:
        return cache
    
    print(f"Haciendo la peticion: {speciality_row['name']}")
    
    speciality_url = speciality_row['link']
    
    r = requests.get(speciality_url)
    bs_content = BeautifulSoup(r.text, 'lxml')

    table = bs_content.find('table', attrs={'id': 'indexlist'})
    rows = table.findChildren('tr')
    
    structured_rows = []
    for row in rows:
        if not row.get('class')[0] in ['even', 'odd']:
            continue
        cols = row.findChildren('td')
        name = cols[1].a.get_text().strip().replace('.', '')
        
        structured = {
            'type': get_type_and_tribunal(name)[0].strip(),
            'tribunal': get_type_and_tribunal(name)[1],
            'name': name,
            'link': urllib.parse.urljoin(speciality_url, cols[1].a.get('href')).strip(),
            'modified': cols[2].get_text().strip(),
        }
        structured_rows.append(structured)

    save_to_json_file(CACHE_FILE, structured_rows)    
    return structured_rows


def get_all_files(structured_row, include_patterns=['ActaNotesProva2Provisional.pdf']):
    slug = slugify(f"files_tribunal_{structured_row['name']}{hash_for_obj(include_patterns)}")
    CACHE_FILE = f'cache/{slug}a.json'
    
    cache = load_json_file(CACHE_FILE)
    if cache:
        return cache
    
    print(f"Haciendo la peticion: {CACHE_FILE}")
    
    url_files = structured_row['link']
    r = requests.get(url_files)
    bs_content = BeautifulSoup(r.text, 'lxml')
    table = bs_content.find('table', attrs={'id': 'indexlist'})
    rows = table.findChildren('tr')
    
    structured_rows = []
    for row in rows:
        if not row.get('class')[0] in ['even', 'odd']:
            continue
        cols = row.findChildren('td')
        name = cols[1].a.get_text().strip()
        
        # Only include expected files
        included = False
        for included_pattern in include_patterns:
            if included_pattern in name:
                included = True

        if not included:
            continue

        structured = {
            '_tribunal': structured_row,
            'name': name,
            'link': urllib.parse.urljoin(url_files, cols[1].a.get('href')),
            'modified': cols[2].get_text().strip(),
        }
        structured_rows.append(structured)

    save_to_json_file(CACHE_FILE, structured_rows)    
    return structured_rows

def download_file(structured_row):
    tribunal_type = structured_row['_tribunal']['type']
    tribunal = structured_row['_tribunal']['tribunal']
    modified = structured_row['modified'].replace(':', '_').replace('-', '_')
    url = structured_row['link']
    name = Path(structured_row['name'])
    name_wo_ext = name.with_suffix('')
    name_ext = name.suffix
    file_name = f"{tribunal} - {name_wo_ext} {modified}{name_ext}"
    final_name = f"{tribunal_type}/{file_name}"
    
    path = Path(final_name)
    if path.is_file():
        # Skip the download if the file exists
        return final_name
    
    r_doc = requests.get(url)
    open(final_name, 'wb').write(r_doc.content)
    print(f"\t{file_name}")

    return final_name


start_time_str = now_in_string()

for speciality in specialties:
    tribunals = get_all_tribunals(speciality)
    print(f"===== START {speciality['name']}")
    
    for idx, tribunal in enumerate(tribunals):
        # Create the directory for storing the PDFs
        tribunal_type = tribunal['type']
        exists = os.path.exists(tribunal_type)
        if not exists:
            os.makedirs(tribunal_type)
        
        counter_str = f"{idx}/{len(tribunals) - 1}"
        print(f"{now_in_string()} ({counter_str}) {tribunal['type']} {tribunal['tribunal']}")
        
        include_patterns=[
            'ActaNotes1Provisional.pdf',
            'ActaNotes1Definitiva.pdf',
            'ActaNotesProva2Provisional.pdf',
            'ActaNotesProva2Definitiva.pdf'
        ]
        files = get_all_files(tribunal, include_patterns=include_patterns)
        
        for file in files:
            download_file(file)
        
    print(f"===== END {speciality['name']}")

print(f"===== END ({start_time_str} -> {now_in_string()}")


In [ ]:
import os
import tabula

CSV_PATH = "./csv/tmp/"
CACHE_FILES = "./cache/tabula/" 

def extract_data_from_pdf_to_csv(file, suffix):
    result = re.search('([\w]*) - ([\w]*)', file.name)
    specialty = str(file.parent).strip()
    tribunal = result[1]    
    
    CACHE_FILE = Path().joinpath(CACHE_FILES, f"{specialty}_{tribunal}_{hash_for_obj(file.name)}.csv")
    path = Path(CACHE_FILE)
    if path.is_file():
        # Skip the download if the file exists
        print(f"Skipping tabula for {file}")
        return
    
    tables = tabula.read_pdf(file,pages="all")
    print(f"({len(tables)} tables on this file)")
    
    for idx, table in enumerate(tables):
        table['specialty'] = [specialty for x in range(len(table))]
        table['tribunal'] = [tribunal for x in range(len(table))]
        filename = Path().joinpath(CSV_PATH, f"{specialty}_{tribunal}_{suffix}_{idx}.csv")
        table.to_csv(filename, index=False)
    
    save_to_json_file(CACHE_FILE, {'date': now_in_string()})

def extract_data_from_pattern(glob_pattern, suffix):
    p = Path('.').glob(glob_pattern)
    files = [x for x in p if x.is_file()]
    for idx, file in enumerate(files):
        counter_str = f"{idx}/{len(files)}"
        print(f"{now_in_string()} ({counter_str}) Reading '{file}'...")
        extract_data_from_pdf_to_csv(file, suffix)

    print("FINISH")


exists = os.path.exists(CACHE_FILES)
if not exists:
    os.makedirs(CACHE_FILES)


# Just to load the methods on this cell without running it
# Create the directory for storing the CSVs
exists = os.path.exists(CSV_PATH)
if not exists:
    os.makedirs(CSV_PATH)

    
include_patterns=[
    ('**/*ActaNotes1Provisional*.pdf', 'provisional-1'),
    ('**/*ActaNotes1Definitiva*.pdf', 'definitiva-1'),
    ('**/*ActaNotesProva2Provisional*.pdf', 'provisional-2'),
    ('**/*ActaNotesProva2Definitiva*.pdf', 'definitiva-2')
]

    
start_time_str = now_in_string()
print(f"===== START {start_time_str}")
for pattern in include_patterns:
    extract_data_from_pattern(pattern[0], suffix=pattern[1])
print(f"===== END ({start_time_str} -> {now_in_string()}")

In [ ]:
import pandas as pd
import glob

# This step requires running a couple of commands manually:
# (it could be included on this jupyter, but sometimes the shell is faster :)
#
#
# cat csv/tmp/*provisional-1*.csv | grep "\*\*\*" > csv/PROVISIONAL1.csv
# cat csv/tmp/*provisional-2*.csv | grep "\*\*\*" > csv/PROVISIONAL2.csv
# cat csv/tmp/*definitiva1-1*.csv | grep "\*\*\*" > csv/DEFINITIVA1.csv
# cat csv/tmp/*definitiva-2*.csv | grep "\*\*\*" > csv/DEFINITIVA2.csv

COLS_PRUEBA1 = ['DNI', 'nombre', 'tema', 'caso', 'total', 'especialidad', 'tribunal']
COLS_PRUEBA2 = ['DNI', 'nombre', 'program', 'especialidad', 'tribunal']

def load_csv_file(filename, cols_names):
    return pd.read_csv(filename, names=cols_names)


def field_is_equal(row, field_name_1, field_name_2):
    str_1 = str(row[field_name_1])
    str_2 = str(row[field_name_2])
    
    return str_1 == str_2


def are_equal_provisional_and_definitive(field_name):
    def return_function(row):
        return field_is_equal(row, f'{field_name}_prov', f'{field_name}_def')
    
    return return_function

def modifying_approval_part_1(row):
    str_1 = str(row['total_prov'])
    str_2 = str(row['total_def'])
    
    if str_1 in ['nan', 'NP'] and str_2 not in ['nan', 'NP']:
        return "aprobado"

    if str_2 in ['nan', 'NP'] and str_1 not in ['nan', 'NP']:
        return "suspenso"
    
    return "-"

def find_duplicated(total_dataset):
    dupli_dict = {}
    
    duplicates = total_dataset[total_dataset.duplicated(subset=['DNI','especialidad', 'tribunal'], keep=False)]
    for idx, dup in duplicates.iterrows():
        # For each duplicated element, I'm going to look for the rest of occurences of that value
        # and compare the name. If the name is partially similar, I'll include in the results for debug them
        name = dup['nombre']

        others = duplicates[(
            (duplicates['DNI'] == dup['DNI'])
            & (duplicates['especialidad'] == dup['especialidad'])
            & (duplicates['tribunal'] == dup['tribunal'])
            & (duplicates['nombre'] != dup['nombre'])
        )]

        for ido, other in others.iterrows():
            name_other = other['nombre']

            if name in name_other:
                dupli_dict[name] = name_other
    return dupli_dict

################

start_time_str = now_in_string()
print(f"===== START {start_time_str}")

provisional_1 = load_csv_file('csv/PROVISIONAL1.csv', COLS_PRUEBA1)
definitiva_1 = load_csv_file('csv/DEFINITIVA1.csv', COLS_PRUEBA1)

provisional_2 = load_csv_file('csv/PROVISIONAL2.csv', COLS_PRUEBA2)
definitiva_2 = load_csv_file('csv/DEFINITIVA2.csv', COLS_PRUEBA2)


prueba_1 = pd.merge(provisional_1, definitiva_1, on=['DNI', 'especialidad', 'tribunal', 'nombre'], how="outer", suffixes=('_prov', '_def'))
prueba_2 = pd.merge(provisional_2, definitiva_2, on=['DNI', 'especialidad', 'tribunal', 'nombre'], how="outer", suffixes=('_prov', '_def'))

total = pd.merge(prueba_1, prueba_2, on=['DNI', 'especialidad', 'tribunal', 'nombre'], how="outer", suffixes=('_1', '_2'))              


duplicados = find_duplicated(total)
print(f"Rarezas encontradas: {len(duplicados)}")

# Fix the names with 'rarezas'
for short_name, long_name in duplicados.items():
    prueba_1.replace(short_name, long_name, inplace=True)
    prueba_2.replace(short_name, long_name, inplace=True)

total_fixed = pd.merge(prueba_1, prueba_2, on=['DNI', 'especialidad', 'tribunal', 'nombre'], how="outer", suffixes=('_1', '_2'))
duplicados_after_fixing = find_duplicated(total_fixed)
print(f"Rarezas pendientes: {len(duplicados_after_fixing)}")


print(f"prueba 1: {len(prueba_1)}, prueba 2: {len(prueba_2)}, mergeado: {len(total_fixed)}")


# Add columns 'iguales' that compares provisional and definitive results
total_fixed['tema_='] = total_fixed.apply(are_equal_provisional_and_definitive('tema'), axis=1)
total_fixed['caso_='] = total_fixed.apply(are_equal_provisional_and_definitive('caso'), axis=1)
total_fixed['total_p1_='] = total_fixed.apply(modifying_approval_part_1, axis=1)
total_fixed['program_='] = total_fixed.apply(are_equal_provisional_and_definitive('program'), axis=1)

# Sort the columns to look better on the spreedsheet
total_fixed = total_fixed.reindex(columns=['especialidad', 'tribunal', 'DNI', 'nombre',
                                           'tema_prov', 'tema_def', 'tema_=',
                                           'caso_prov', 'caso_def', 'caso_=',
                                           'total_prov', 'total_def', 'total_p1_=',
                                           'program_prov', 'program_def', 'program_='
                                          ])

fila = total_fixed[total_fixed['total_p1_='] == 'suspenso']
fila

print(f"===== END ({start_time_str} -> {now_in_string()})")

In [ ]:
start_time_str = now_in_string()
print(f"===== START {start_time_str}")

# Delete the name column (for sharing):
del total_fixed['nombre']

with pd.ExcelWriter('results_sin_nombre.xlsx') as writter:
    for speciality in total_fixed['especialidad'].unique():
        temp_df = total_fixed.loc[total_fixed['especialidad'] == speciality]
        del temp_df['especialidad']
        temp_df.to_excel(writter, sheet_name=speciality, index=False)


print(f"===== END ({start_time_str} -> {now_in_string()})")